In [15]:
import gym
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow import keras
import os
import datetime
from gym import wrappers
from particle_envs.make_env import make_env

In [16]:
class MyModel(tf.keras.Model):
    def __init__(self, num_states, hidden_units, num_actions):
        super(MyModel, self).__init__()
        self.input_layer = tf.keras.layers.InputLayer(input_shape=(num_states,))
        self.hidden_layers = []
        for output in hidden_units:
            self.hidden_layers.append(Dense(output, activation='tanh', kernel_initializer='RandomNormal'))
        self.output_layer = Dense(num_actions, activation='linear', kernel_initializer='RandomNormal')
        
    @tf.function
    def call(self, inputs):
        z = self.input_layer(inputs)
        for layer in self.hidden_layers:
            z = layer(z)
        output = self.output_layer(z)
        return output

In [17]:
class DQN:
    def __init__(self, num_states, num_actions, hidden_units, gamma, max_experiences, min_experiences, batch_size, lr):
        self.num_actions = num_actions
        self.batch_size = batch_size
        self.optimizer = tf.optimizers.Adam(lr)
        self.gamma = gamma
        self.model = MyModel(num_states, hidden_units, num_actions)
        self.experience = {'s': [], 'a': [], 'r': [], 's2': [], 'done': []}  # done (bool): is current state terminal
        self.max_experiences = max_experiences
        self.min_experiences = min_experiences
         
    def predict(self, inputs):
        return self.model(np.atleast_2d(inputs.astype('float32')))
    
    def train(self, TargetNet):
        if len(self.experience['s']) < self.min_experiences:
            return 0, 0
        ids = np.random.randint(low=0, high=len(self.experience['s']), size=self.batch_size)
        states = np.asarray([self.experience['s'][i] for i in ids])
        actions = np.asarray([self.experience['a'][i] for i in ids])
        rewards = np.asarray([self.experience['r'][i] for i in ids])
        states_next = np.asarray([self.experience['s2'][i] for i in ids])
        dones = np.asarray([self.experience['done'][i] for i in ids])
        value_next = np.max(TargetNet.predict(states_next), axis=1)
        actual_values = np.where(dones, rewards, rewards + self.gamma*value_next)
        
        with tf.GradientTape() as tape:
            selected_action_values = tf.math.reduce_sum(
                self.predict(states) * actions)
            loss = tf.math.reduce_mean(tf.square(actual_values - selected_action_values))
    
        variables = self.model.trainable_variables
        gradients = tape.gradient(loss, variables)
        self.optimizer.apply_gradients(zip(gradients, variables))
        return loss, selected_action_values
        
    def get_action(self, states, epsilon):
        if np.random.random() < epsilon:
            return np.random.choice(self.num_actions)
        else:
            return np.argmax(self.predict(np.atleast_2d(states))[0])


    def add_experience(self, exp):
        if len(self.experience['s']) >= self.max_experiences:
            for key in self.experience.keys():
                self.experience[key].pop(0)
        for key, value in exp.items():
            self.experience[key].append(value)

    def copy_weights(self, TrainNet):
        variables1 = self.model.trainable_variables
        variables2 = TrainNet.model.trainable_variables
        for v1, v2 in zip(variables1, variables2):
            v1.assign(v2.numpy())

In [18]:
def play_game(env, TrainNet, TargetNet, epsilon, copy_step):
    rewards = 0
    iter = 0
    done = False
    observations = env.reset()[0]
    losses = list()
    episode_length = 200
    for i in range(episode_length):
#         env.render()
        action = TrainNet.get_action(observations, epsilon)
        # Convert action to one-hot
        action = [np.identity(env.action_space[0].n, dtype=int)[action]]
        prev_observations = observations
        observations, reward, done, _ = env.step(action)
        observations, reward, done = observations[0], reward[0], done[0]
        rewards += reward
#         if done:
#             reward = -200
#             env.reset()

        exp = {'s': prev_observations, 'a': action, 'r': reward, 's2': observations, 'done': done}
        TrainNet.add_experience(exp)
        loss, selected_action_values = TrainNet.train(TargetNet)
        if isinstance(loss, int):
            losses.append(loss)
        else:
            losses.append(loss.numpy())
        iter += 1
        if iter % copy_step == 0:
            TargetNet.copy_weights(TrainNet)
    return rewards, np.mean(losses), np.mean(selected_action_values)

In [19]:
def make_video(env, TrainNet):
#     env = wrappers.Monitor(env, os.path.join(os.getcwd(), "videos"), force=True)
    rewards = 0
    steps = 0
    done = False
    observation = env.reset()
    observation = observation[0]
    for i in range(100):
#         env.render()
        action = TrainNet.get_action(observation, 0)
        # Convert action to one-hot
        action = [np.identity(env.action_space[0].n, dtype=int)[action]]
        observation, reward, done, _ = env.step(action)
        observation, reward, done = observation[0], reward[0], done[0]
        steps += 1
        rewards += reward
    print("Testing steps: {} rewards {}: ".format(steps, rewards))

In [20]:
def main():
    env = make_env('simple')
    gamma = 0.99
    copy_step = 25
    num_states = len(env.observation_space[0].sample())
    num_actions = env.action_space[0].n
    hidden_units = [200, 200]
    max_experiences = 10000
    min_experiences = 100
    batch_size = 32
    lr = 1e-2
    current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    log_dir = 'logs/dqn/' + current_time
    summary_writer = tf.summary.create_file_writer(log_dir)
    TrainNet = DQN(num_states, num_actions, hidden_units, gamma, max_experiences, min_experiences, batch_size, lr)
    TargetNet = DQN(num_states, num_actions, hidden_units, gamma, max_experiences, min_experiences, batch_size, lr)
    N = 5
    total_rewards = np.empty(N)
    epsilon = 0.99
    decay = 0.9999
    min_epsilon = 0.1
    for n in range(N):
        epsilon = max(min_epsilon, epsilon * decay)
        total_reward, losses, action_values_avg = play_game(env, TrainNet, TargetNet, epsilon, copy_step)
        total_rewards[n] = total_reward
        avg_rewards = total_rewards[max(0, n - 100):(n + 1)].mean()
        with summary_writer.as_default():
            tf.summary.scalar('episode reward', total_reward, step=n)
            tf.summary.scalar('running avg reward(100)', avg_rewards, step=n)
            tf.summary.scalar('average loss)', losses, step=n)
            tf.summary.scalar('average Q value', action_values_avg, step=n)
        if True:  # n % 100 == 0:
            print("episode:", n, "episode reward:", total_reward, "epsilon:", epsilon, "avg reward (last 100):", avg_rewards,
                  "episode loss: ", losses)
    print("avg reward for last 100 episodes:", avg_rewards)
    make_video(env, TrainNet)
    env.close()
    TrainNet.model.save('.\dqn_model\model_{}'.format(current_time))

In [21]:
if __name__ == '__main__':
    main()

episode: 0 episode reward: -610.391213194183 epsilon: 0.989901 avg reward (last 100): -610.391213194183 episode loss:  1996.7965217283368
episode: 1 episode reward: -1411.7662770719487 epsilon: 0.9898020099 avg reward (last 100): -1011.0787451330659 episode loss:  101.5406
episode: 2 episode reward: -1145.972213335993 epsilon: 0.98970302969901 avg reward (last 100): -1056.0432345340416 episode loss:  25.634985
episode: 3 episode reward: -117.42872119214348 epsilon: 0.9896040593960401 avg reward (last 100): -821.3896061985672 episode loss:  30.445684
episode: 4 episode reward: -335.1338048245317 epsilon: 0.9895050989901005 avg reward (last 100): -724.1384459237601 episode loss:  25.070498
avg reward for last 100 episodes: -724.1384459237601
Testing steps: 100 rewards -77.10638315460645: 
INFO:tensorflow:Assets written to: .\dqn_model\model_20200711-180057\assets
